In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 🌱 Reveal Plant - Model Inference\n",
        "Bu notebook GitHub Actions üzerinden tetiklenerek model tahminini yapar."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "execution": {
          "iopub.execute_input": "2024-01-01T00:00:00.000000Z",
          "iopub.status.busy": "2024-01-01T00:00:00.000000Z",
          "iopub.status.idle": "2024-01-01T00:00:00.000000Z",
          "shell.execute_reply": "2024-01-01T00:00:00.000000Z"
        }
      },
      "outputs": [],
      "source": [
        "import os\n",
        "import json\n",
        "import numpy as np\n",
        "from pathlib import Path\n",
        "from tensorflow import keras\n",
        "from tensorflow.keras.preprocessing import image\n",
        "\n",
        "print(\"✓ Kütüphaneler yüklendi\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Model yolunu belirle\n",
        "model_path = \"/kaggle/input/resnet101/keras/default/1/PlantVillage_Resnet101_FineTuning.keras\"\n",
        "\n",
        "# Modeli yükle\n",
        "print(f\"Model yükleniyor: {model_path}\")\n",
        "model = keras.models.load_model(model_path)\n",
        "print(\"✓ Model başarıyla yüklendi\")\n",
        "\n",
        "# Model mimarisini göster\n",
        "model.summary()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Sınıf etiketleri (PlantVillage dataset - 38 sınıf)\n",
        "# NOT: Kendi sınıflarınızı buraya ekleyin\n",
        "class_names = [\n",
        "    'Apple___Apple_scab',\n",
        "    'Apple___Black_rot',\n",
        "    'Apple___Cedar_apple_rust',\n",
        "    'Apple___healthy',\n",
        "    'Blueberry___healthy',\n",
        "    'Cherry_(including_sour)___Powdery_mildew',\n",
        "    'Cherry_(including_sour)___healthy',\n",
        "    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',\n",
        "    'Corn_(maize)___Common_rust_',\n",
        "    'Corn_(maize)___Northern_Leaf_Blight',\n",
        "    'Corn_(maize)___healthy',\n",
        "    'Grape___Black_rot',\n",
        "    'Grape___Esca_(Black_Measles)',\n",
        "    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',\n",
        "    'Grape___healthy',\n",
        "    'Orange___Haunglongbing_(Citrus_greening)',\n",
        "    'Peach___Bacterial_spot',\n",
        "    'Peach___healthy',\n",
        "    'Pepper,_bell___Bacterial_spot',\n",
        "    'Pepper,_bell___healthy',\n",
        "    'Potato___Early_blight',\n",
        "    'Potato___Late_blight',\n",
        "    'Potato___healthy',\n",
        "    'Raspberry___healthy',\n",
        "    'Soybean___healthy',\n",
        "    'Squash___Powdery_mildew',\n",
        "    'Strawberry___Leaf_scorch',\n",
        "    'Strawberry___healthy',\n",
        "    'Tomato___Bacterial_spot',\n",
        "    'Tomato___Early_blight',\n",
        "    'Tomato___Late_blight',\n",
        "    'Tomato___Leaf_Mold',\n",
        "    'Tomato___Septoria_leaf_spot',\n",
        "    'Tomato___Spider_mites Two-spotted_spider_mite',\n",
        "    'Tomato___Target_Spot',\n",
        "    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',\n",
        "    'Tomato___Tomato_mosaic_virus',\n",
        "    'Tomato___healthy'\n",
        "]\n",
        "\n",
        "print(f\"✓ Toplam {len(class_names)} sınıf yüklendi\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def preprocess_image(img_path, target_size=(224, 224)):\n",
        "    \"\"\"Görseli model için hazırla\"\"\"\n",
        "    img = image.load_img(img_path, target_size=target_size)\n",
        "    img_array = image.img_to_array(img)\n",
        "    img_array = np.expand_dims(img_array, axis=0)\n",
        "    img_array = img_array / 255.0  # Normalizasyon\n",
        "    return img_array\n",
        "\n",
        "def predict_image(img_path, model, class_names, top_k=5):\n",
        "    \"\"\"Görsel tahmini yap\"\"\"\n",
        "    try:\n",
        "        # Görseli hazırla\n",
        "        processed_img = preprocess_image(img_path)\n",
        "        \n",
        "        # Tahmin yap\n",
        "        predictions = model.predict(processed_img, verbose=0)\n",
        "        \n",
        "        # En yüksek olasılıklı sınıfları al\n",
        "        top_indices = np.argsort(predictions[0])[-top_k:][::-1]\n",
        "        top_predictions = [\n",
        "            {\n",
        "                'class': class_names[idx],\n",
        "                'confidence': float(predictions[0][idx])\n",
        "            }\n",
        "            for idx in top_indices\n",
        "        ]\n",
        "        \n",
        "        return {\n",
        "            'success': True,\n",
        "            'image': str(img_path),\n",
        "            'top_prediction': top_predictions[0]['class'],\n",
        "            'top_confidence': top_predictions[0]['confidence'],\n",
        "            'top_k_predictions': top_predictions\n",
        "        }\n",
        "    \n",
        "    except Exception as e:\n",
        "        return {\n",
        "            'success': False,\n",
        "            'image': str(img_path),\n",
        "            'error': str(e)\n",
        "        }\n",
        "\n",
        "print(\"✓ Tahmin fonksiyonları tanımlandı\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Test görsellerini bul\n",
        "working_dir = Path('/kaggle/working')\n",
        "test_images = list(working_dir.glob('test_image.*'))\n",
        "\n",
        "if not test_images:\n",
        "    print(\"⚠️ Test görseli bulunamadı!\")\n",
        "    print(\"Mevcut dosyalar:\")\n",
        "    print(list(working_dir.iterdir()))\n",
        "else:\n",
        "    print(f\"✓ {len(test_images)} test görseli bulundu\")\n",
        "    for img in test_images:\n",
        "        print(f\"  - {img.name}\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Tahminleri yap\n",
        "results = []\n",
        "\n",
        "for img_path in test_images:\n",
        "    print(f\"\\n{'='*60}\")\n",
        "    print(f\"Tahmin yapılıyor: {img_path.name}\")\n",
        "    print('='*60)\n",
        "    \n",
        "    result = predict_image(img_path, model, class_names, top_k=5)\n",
        "    results.append(result)\n",
        "    \n",
        "    if result['success']:\n",
        "        print(f\"✓ En olası: {result['top_prediction']}\")\n",
        "        print(f\"  Güven: {result['top_confidence']*100:.2f}%\")\n",
        "        print(f\"\\n  Top 5 Tahmin:\")\n",
        "        for i, pred in enumerate(result['top_k_predictions'], 1):\n",
        "            print(f\"    {i}. {pred['class']}: {pred['confidence']*100:.2f}%\")\n",
        "    else:\n",
        "        print(f\"✗ Hata: {result['error']}\")\n",
        "\n",
        "print(f\"\\n✓ Toplam {len(results)} görsel işlendi\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Sonuçları JSON olarak kaydet\n",
        "output_file = working_dir / 'predictions.json'\n",
        "\n",
        "with open(output_file, 'w', encoding='utf-8') as f:\n",
        "    json.dump(results, f, indent=2, ensure_ascii=False)\n",
        "\n",
        "print(f\"✓ Sonuçlar kaydedildi: {output_file}\")\n",
        "print(f\"\\n📊 ÖZET:\")\n",
        "print(f\"  Başarılı: {sum(1 for r in results if r['success'])}\")\n",
        "print(f\"  Başarısız: {sum(1 for r in results if not r['success'])}\")"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.10.12"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 4
}